In [ ]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from scipy import signal
import IPython
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (16,8)

In [ ]:
dat = np.fromfile("/home/faux/a.sr2048000.cu8", dtype=np.uint8).astype(np.float32, casting='safe').view(np.complex64)
SR = 2048000

In [ ]:
def spec(dat, sr=SR):
    plt.specgram(dat, NFFT=32, noverlap=8, Fs=sr)

def freq(dat):
    plt.psd(dat, NFFT=1024, Fs=SR)

def tone(freq, samples, sr=SR):
    return np.exp(-2j*np.pi*-freq/sr*np.arange(samples))

In [ ]:
cut = dat[int(0.62*SR):int((0.62+0.079)*SR)];
spec(cut)

In [ ]:
def deci(wat, by):
    return signal.decimate(wat, by, zero_phase=False)

In [ ]:
chan = deci(cut*tone(-200e3, len(cut)), 16)

# TODO: no idea where this 400k comes from, it's should be 25k, which is off by 16.
left = deci(chan*tone(-410e3, len(chan), sr=SR//16), 4)
right = deci(chan*tone(410e3/16, len(chan), sr=SR//16), 4)

plt.subplot(311); spec(chan, sr=SR//16)
plt.subplot(312); spec(left, sr=SR//16//4)
plt.subplot(313); spec(right, sr=SR//16//4)

In [ ]:
def w(l, n):
    for i in range(0, len(l), n // 8):
        yield l[i:i + n]

dat = list(x-y for (x,y) in zip(
    (sum(abs(x)) for x in w(left, 8)),
    (sum(abs(x)) for x in w(right, 8))
))
_ = plt.plot(dat)

In [ ]:
POW = 200
''.join('1' if x > POW else '0' if x < -POW else '_' for x in dat)

In [ ]:
SYM = 13

def deco(val):
    cur = False
    lo = 0
    hi = 0
    un = 0
    for item in val:
        if item < -POW:
            if cur:
                for i in range(round((hi+un)/SYM)):
                    yield cur
                #yield (lo, hi, un)
                lo = 0; hi = 0; un = 0
                cur = False
            else:
                lo += 1
        elif item > POW:
            if not cur:
                for i in range(round((lo+un)/SYM)):
                    yield cur
                #yield (lo, hi, un)
                lo = 0; hi = 0; un = 0
                cur = True
            else:
                hi += 1
        else:
            un += 1

man = list(deco(dat))
_ = plt.stem(man[53:])

In [ ]:
def deman(arr):
    for i in range(0, len(arr), 2):
        pair = arr[i:i+2]
        if [True, False] == pair:
            yield '0'
        elif [False, True] == pair:
            yield '1'
        else:
            return

bits = ''.join(deman(man[52:]))
bits

In [ ]:
def slide(x):
    for i in range(0, len(x), 8):
        yield x[i:i+8]

[''.join(chr(int(x, 2)) for x in slide(bits[s:])) for s in range(8)]